<a href="https://colab.research.google.com/github/danbom/tensorflow/blob/master/Lab06_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 06 Softmax Zoo_classifier-eager

In [ ]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,-1]

nb_classes = 7

Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

In [ ]:
!ls -l

In [ ]:
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name = 'bias')
variables = [W,b]

def logit_fn(X):
    return tf.matmul(X,W) + b

def hypothesis(X) :
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X,Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits,from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_fn(X,Y):
    with tf.GradientTape() as tape :
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X,Y) :
    pred = tf.argmax(hypothesis(X),1)
    correct_prediction = tf.equal(pred, tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [ ]:
def fit(X,Y,epochs=1000,verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs) :
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X,Y).numpy()
            loss = cost_fn(X,Y).numpy()
            print('Step: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

- SGD : 확률적 경사하강법 Stochastic Gradient Descent